In [3]:
// prio_hol_closed example - multiclass with PS, SIRO, FCFS, HOL priority
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*

In [4]:
// Create network
val model = Network("MyNetwork")

// Block 1: nodes
val node1 = Delay(model, "SlowDelay")
val node2 = Queue(model, "FCFSQueue", SchedStrategy.FCFS)
val node3 = Queue(model, "SIROQueue", SchedStrategy.SIRO)
val node4 = Queue(model, "PSQueue", SchedStrategy.PS)
val node5 = Queue(model, "HOLQueue", SchedStrategy.HOL)
val node6 = Delay(model, "FastDelay")

In [5]:
// Block 2: classes
val jobclass1 = ClosedClass(model, "Class1", 18, node1, 0)
val jobclass2 = ClosedClass(model, "Class2", 18, node1, 1)
val jobclass3 = ClosedClass(model, "Class3", 18, node1, 0)

In [6]:
// Set service times for all nodes and classes
node1.setService(jobclass1, Exp.fitMean(10.0))
node1.setService(jobclass2, Exp.fitMean(10.0))
node1.setService(jobclass3, Exp.fitMean(10.0))

node2.setService(jobclass1, Exp.fitMean(0.3))
node2.setService(jobclass2, Exp.fitMean(0.5))
node2.setService(jobclass3, Exp.fitMean(0.6))

node3.setService(jobclass1, Exp.fitMean(1.1))
node3.setService(jobclass2, Exp.fitMean(1.3))
node3.setService(jobclass3, Exp.fitMean(1.5))

node4.setService(jobclass1, Exp.fitMean(1.0))
node4.setService(jobclass2, Exp.fitMean(1.1))
node4.setService(jobclass3, Exp.fitMean(1.9))

node5.setService(jobclass1, Exp.fitMean(2.5))
node5.setService(jobclass2, Exp.fitMean(1.9))
node5.setService(jobclass3, Exp.fitMean(4.3))

node6.setService(jobclass1, Exp.fitMean(1.0))
node6.setService(jobclass2, Exp.fitMean(1.0))
node6.setService(jobclass3, Exp.fitMean(1.0))

In [7]:
// Block 3: topology - initialize routing matrix
val P = model.initRoutingMatrix()

// Class1 routing
P.set(jobclass1, jobclass1, node1, node2, 1.0) // SlowDelay -> FCFSQueue
P.set(jobclass1, jobclass1, node2, node3, 0.25) // FCFSQueue -> SIROQueue
P.set(jobclass1, jobclass1, node2, node4, 0.25) // FCFSQueue -> PSQueue
P.set(jobclass1, jobclass1, node2, node5, 0.25) // FCFSQueue -> HOLQueue
P.set(jobclass1, jobclass1, node2, node6, 0.25) // FCFSQueue -> FastDelay
P.set(jobclass1, jobclass1, node3, node2, 1.0) // SIROQueue -> FCFSQueue
P.set(jobclass1, jobclass1, node4, node2, 1.0) // PSQueue -> FCFSQueue
P.set(jobclass1, jobclass1, node5, node2, 1.0) // HOLQueue -> FCFSQueue
P.set(jobclass1, jobclass1, node6, node1, 1.0) // FastDelay -> SlowDelay

// Class2 routing
P.set(jobclass2, jobclass2, node1, node2, 1.0) // SlowDelay -> FCFSQueue
P.set(jobclass2, jobclass2, node2, node3, 0.25) // FCFSQueue -> SIROQueue
P.set(jobclass2, jobclass2, node2, node4, 0.25) // FCFSQueue -> PSQueue
P.set(jobclass2, jobclass2, node2, node5, 0.25) // FCFSQueue -> HOLQueue
P.set(jobclass2, jobclass2, node2, node6, 0.25) // FCFSQueue -> FastDelay
P.set(jobclass2, jobclass2, node3, node2, 1.0) // SIROQueue -> FCFSQueue
P.set(jobclass2, jobclass2, node4, node2, 1.0) // PSQueue -> FCFSQueue
P.set(jobclass2, jobclass2, node5, node2, 1.0) // HOLQueue -> FCFSQueue
P.set(jobclass2, jobclass2, node6, node1, 1.0) // FastDelay -> SlowDelay

// Class3 routing
P.set(jobclass3, jobclass3, node1, node2, 1.0) // SlowDelay -> FCFSQueue
P.set(jobclass3, jobclass3, node2, node3, 0.25) // FCFSQueue -> SIROQueue
P.set(jobclass3, jobclass3, node2, node4, 0.25) // FCFSQueue -> PSQueue
P.set(jobclass3, jobclass3, node2, node5, 0.25) // FCFSQueue -> HOLQueue
P.set(jobclass3, jobclass3, node2, node6, 0.25) // FCFSQueue -> FastDelay
P.set(jobclass3, jobclass3, node3, node2, 1.0) // SIROQueue -> FCFSQueue
P.set(jobclass3, jobclass3, node4, node2, 1.0) // PSQueue -> FCFSQueue
P.set(jobclass3, jobclass3, node5, node2, 1.0) // HOLQueue -> FCFSQueue
P.set(jobclass3, jobclass3, node6, node1, 1.0) // FastDelay -> SlowDelay

model.link(P)

In [8]:
// Solve with different solvers
MVA(model, "seed", 23000).avgTable.print()
JMT(model, "seed", 23000, "samples", 10000).avgTable.print()

Station    JobClass  QLen        Util        RespT       ResidT      ArvR        Tput        
---------------------------------------------------------------------------------------------
SlowDelay  Class1    1.88242     1.88242     10.00000    10.00000    0.18824     0.18824     
SlowDelay  Class2    2.33322     2.33322     10.00000    10.00000    0.23332     0.23332     
SlowDelay  Class3    1.21982     1.21982     10.00000    10.00000    0.12198     0.12198     
FCFSQueue  Class1    10.44612    0.22589     13.87324    55.49297    0.75297     0.75297     
FCFSQueue  Class2    12.96088    0.46664     13.88732    55.54929    0.93329     0.93329     
FCFSQueue  Class3    6.88845     0.29276     14.11773    56.47092    0.48793     0.48793     
SIROQueue  Class1    0.72594     0.20707     3.85640     3.85640     0.18824     0.18824     
SIROQueue  Class2    0.94094     0.30332     4.03281     4.03281     0.23332     0.23332     
SIROQueue  Class3    0.51934     0.18297     4.25749     4.2